# RAG로 AI 소믈리에 - wine pairing

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI

## LLM을 통한 요리 이미지 -> 맛 & 풍미 (image to text)

In [6]:
# image에서 맛(풍미)에 대한 설명 text 생성
def describe_dish_flavor(query):
    # pass
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona: You are a highly skilled and perceptive culinary expert with a deep understanding of flavors, aromas, and textures in a wide variety of cuisines. Your personality is professional, insightful, and approachable, dedicated to helping users understand and appreciate the complexities of taste in their food experiences. You are passionate about exploring subtle nuances in ingredients and dishes, making flavor analysis accessible and engaging.

            Role: Your role is to guide users in understanding and analyzing the taste, aroma, texture, and overall flavor profile of various foods. You provide detailed descriptions of flavors and offer insights into how different ingredients, cooking techniques, and seasonings influence the dish's final taste. You also help users make informed choices about ingredient combinations and cooking methods to achieve desired flavors in their culinary creations.
            Examples:
            Flavor Profile Analysis: If a user describes a dish with grilled lamb seasoned with rosemary and garlic, you might explain how the earthy, woody notes of rosemary enhance the rich, savory flavor of the lamb. You could also describe how the caramelization from grilling adds a layer of smokiness, balanced by the mild sweetness of roasted garlic.
            Texture and Mouthfeel Explanation: If a user is tasting a creamy mushroom risotto, you might highlight the importance of the dish’s creamy, velvety texture achieved through the slow release of starch from Arborio rice. You could also mention how the umami-rich flavor of mushrooms adds depth to the dish, while the cheese provides a slight saltiness that balances the creaminess.
            Pairing Suggestions: If a user is preparing a spicy Thai green curry, you could recommend balancing its heat with a slightly sweet or acidic side, such as a cucumber salad or coconut rice. You might explain how the coolness of cucumber contrasts with the curry’s heat, and how the subtle sweetness in coconut rice tempers the dish’s spiciness, creating a harmonious dining experience.
         """),
        ("human", """
            이미지의 요리명과 풍미를 한 문장으로 요약해 주세요.
        """)
    ])

    # image url list
    template = []

    if query.get("images_urls"):
        template += [{"image_url":image_url} for image_url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=4095)

    chain = prompt | llm | StrOutputParser()

    return chain

In [7]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(describe_dish_flavor)

In [8]:
url = 'https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg'
res = r1.invoke({
    "image_urls":[url]
})

print(res)

이미지를 제공해 주시면 요리명과 풍미를 분석하여 요약해 드리겠습니다. 이미지가 없으시다면, 요리에 대한 설명을 주시면 그에 맞춰 풍미를 분석해 드릴 수 있습니다.


## 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

embedding = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
vector_store = PineconeVectorStore(
    index_name=os.getenv("PINECONE_INDEX_NAME"),
    embedding=embedding,
    namespace=os.getenv("PINECONE_NAMESPACE")
)

query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."
vector_store.similarity_search(query, namespace=os.getenv("PINECONE_NAMESPACE"), k=5)

[Document(id='dbb45957-40df-490a-b7ab-985c5fcbd120', metadata={'row': 20494.0, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=': 20494\ncountry: US\ndescription: This lightly sweet wine has an intense oaky, peppery, smoky aroma and the flavors come out the same way. The beefy, liquid-smoke character should match ribs in barbecue sauce.\ndesignation: Sultry\npoints: 85\nprice: 12.0\nprovince: California\nregion_1: California\nregion_2: California Other\ntaster_name: Jim Gordon\ntaster_twitter_handle: @gordone_cellars\ntitle: Hey Mambo 2013 Sultry Red (California)\nvariety: Red Blend\nwinery: Hey Mambo'),
 Document(id='f4356985-1b31-4b2d-8848-5771f2bcc118', metadata={'row': 10514.0, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=': 10514\ncountry: US\ndescription: Very strongly flavored, and needs a salty cut of meat to tame its powers. Explodes with jammy blackberries, blueberries, cherries and raspberries, with a monster blast of black pepper. Br

In [10]:
def search_wines(dish_flavor):
    embedding=OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
    vector_store = PineconeVectorStore(
        index_name=os.getenv("PINECONE_INDEX_NAME"),
        embedding=embedding,
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    results = vector_store.similarity_search(dish_flavor, namespace=os.getenv("PINECONE_NAMESPACE"), k=5)

    return {
        "dish_flavor":dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

## RunnableLambda로 실행

In [11]:
r2 = RunnableLambda(search_wines)
query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."
res = r2.invoke(query)
print(res.get("dish_flavor"))
print(res.get("wine_reviews"))

이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다.
: 20494
country: US
description: This lightly sweet wine has an intense oaky, peppery, smoky aroma and the flavors come out the same way. The beefy, liquid-smoke character should match ribs in barbecue sauce.
designation: Sultry
points: 85
price: 12.0
province: California
region_1: California
region_2: California Other
taster_name: Jim Gordon
taster_twitter_handle: @gordone_cellars
title: Hey Mambo 2013 Sultry Red (California)
variety: Red Blend
winery: Hey Mambo
: 10514
country: US
description: Very strongly flavored, and needs a salty cut of meat to tame its powers. Explodes with jammy blackberries, blueberries, cherries and raspberries, with a monster blast of black pepper. Bring on the sausages.
designation: Ex Anima
points: 88
price: 25.0
province: California
region_1: Monterey
region_2: Central Coast
taster_name: 
taster_twitter_handle: 
title: Wrath 2010 Ex Anima Syrah (Monterey)
variety: Syrah
winery: Wrath
: 

## r1, r2로 구성

In [12]:
image_url = 'https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg'
chain = r1 | r2
res = chain.invoke({
    "image_urls":[image_url]
})

In [13]:
print(res.get("dish_flavor"))
print(res.get("wine_reviews"))

이미지를 볼 수 없지만, 요리의 이름과 풍미를 설명해 주시면 그에 대한 분석을 도와드릴 수 있습니다. 요리의 주요 재료나 조리법을 알려주시면, 그에 따른 맛과 향, 질감에 대해 설명해 드리겠습니다.
: 37787
country: US
description: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.
designation: Meiomi
points: 87
price: 25.0
province: California
region_1: Sonoma County-Monterey County-Santa Barbara County
region_2: California Other
taster_name: 
taster_twitter_handle: 
title: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)
variety: Pinot Noir
winery: Belle Glos
: 37405
country: US
description: Tea leaf, cherry and sweet vermouth aromas precede lightly tart cherry, rhubarb and herb flavors in this lean, moderately tannic and un-oaky wine. It's appealing and appetizing in an austere way, giving just enough satisfaction.
designation: Moonglow
points: 88
p

## 와인 추천

In [14]:
def recommend_wines(query):
    prompt = ChatPromptTemplate.from_messages([
           ("system", """
            Persona: You are a refined and approachable virtual wine sommelier with a deep passion for wines, dedicated to helping users explore and enjoy the world of wine with confidence. Your personality is warm, insightful, and patient, ensuring that users feel at ease while learning about wine, regardless of their experience level.
            Role: Your role is to guide users in selecting wines, pairing them with food, and understanding wine characteristics. You are adept at explaining complex wine concepts such as tannins, acidity, and terroir in a way that is accessible to everyone. In addition, you provide suggestions based on the user’s preferences, budget, and the occasion, helping them find the perfect wine to enhance their dining experience.
            Examples:
            Wine Pairing Recommendation: If a user is preparing a buttery garlic shrimp dish, you might suggest a crisp, mineral-driven Chablis or a New Zealand Sauvignon Blanc, explaining how these wines’ acidity and minerality balance the richness of the butter and complement the flavors of the shrimp.
            Wine Selection for a Casual Gathering: If a user is hosting a casual gathering and needs an affordable, crowd-pleasing wine, you might recommend a fruit-forward Pinot Noir or a light Italian Pinot Grigio. Highlight the wines' versatility and how they pair well with a variety of foods, making them ideal for social settings.
            Wine Terminology Explanation: If a user asks what “terroir” means, you would explain it as the unique combination of soil, climate, and landscape in a wine-growing region that influences the wine's flavor, making each wine distinctive to its origin.
            """
         ),
        ("human", """
            와인 페어링 추천에 아래의 요리와 풍미, 와인 리뷰만을 참고하여 한글로 답변해 주세요.

            요리와 풍미:
            {dish_flavor}
         
            와인 리뷰:
            {wine_reviews}
         
            답변은 다음과 같은 값으로 json 데이터로 리턴해주세요.
            recommend_wine :
            recommend_reason :
        """)     
    ])

    llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=4095)
    chain = prompt | llm | JsonOutputParser()

    return chain

In [15]:
r1 = RunnableLambda(describe_dish_flavor)
r2 = RunnableLambda(search_wines)
r3 = RunnableLambda(recommend_wines)

chain = r1 | r2 | r3
img_urls = ['https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg']
res = chain.invoke({
    "image_urls":img_urls
})

In [16]:
print(res)

{'recommend_wine': 'Belle Glos 2008 Meiomi Pinot Noir', 'recommend_reason': '이 와인은 캔디드 라즈베리와 체리, 코코아, 시나몬, 바닐라의 풍미가 조화를 이루며, 스모키한 샌들우드의 달콤한 베니어가 더해져 있습니다. 이러한 풍미는 다양한 요리와 잘 어울리며, 특히 풍부한 소스나 향신료가 가미된 요리와 잘 맞습니다. 또한, 지금 마시기에 적합한 맛을 가지고 있어, 즉시 즐기기에 좋은 선택입니다.'}
